In [1]:
import pygame  # 1. pygame 선언
import random

pygame.init()  # 2. pygame 초기화

# 3. pygame에 사용되는 전역변수 선언
BLACK = (0, 0, 0)
size = [1100, 700]
screen = pygame.display.set_mode(size)
done = False
clock = pygame.time.Clock()
game_font = pygame.font.Font(None, 40)
pygame.display.set_caption("Super Mario")

bgm1_sound = pygame.mixer.Sound("bgm1.wav")
bgm1_sound.play(-1)

# 배경 이미지 불러오기
bgi = pygame.image.load('bgimg.png')
bg2 = pygame.image.load("world2.png")
bg2 = pygame.transform.scale(bg2,(600,700))
bg3 = pygame.image.load("world3.png")
bg3 = pygame.transform.scale(bg3,(600,700))
bg4 = pygame.image.load("world4.png")
bg4 = pygame.transform.scale(bg4,(600,700))
bg5 = pygame.image.load("world5.png")
bg5 = pygame.transform.scale(bg5,(600,700))
bg = [bg2,bg3,bg4,bg5]

# 속도, 질량
VELOCITY = 7
MASS = 2
 
class Mario:
    def __init__(self):
        self.person_image = ""
        self.dx = 0
        self.dy = 0
        self.rect = ""
        self.isJump = 0
        self.v = VELOCITY # 속도
        self.m = MASS  # 질량
    
    # 마리오 만들기
    def load_mario(self):
        self.person_image = pygame.image.load('mario.png')
        # 크기 조정
        self.person_image = pygame.transform.scale(self.person_image, (60, 80))
        self.rect = self.person_image.get_rect()
        self.person = pygame.Rect(self.rect)
        self.person.left = (size[0]-500) // 2 - self.person.width // 2
        self.person.bottom = size[1]
        
    # 마리오 화면띄우기
    def draw_mario(self):
        screen.blit(self.person_image, [self.person.left, self.person.bottom-80])

    # x 좌표 이동
    def move_x(self):
        self.person.left += self.dx
    
    # 점프 메서드
    def jump(self, j):
        self.isJump = j
    
    def update(self):

        if self.isJump > 0:      
            if self.isJump == 2:
                self.v = VELOCITY
            if self.v > 0:
                F = (0.5 * self.m * (self.v * self.v))
            else:
                F = -(0.5 * self.m * (self.v * self.v))

            self.person.bottom -= round(F)
            self.v -= 1

            if self.person.bottom > size[1]:
                self.person.bottom = size[1]
                self.isJump = 0
                self.v = VELOCITY

    def check_screen(self):
        if self.person.left > size[0]-500 or self.person.left < 0:
            self.person.left -= self.dx
        if self.rect.bottom > size[1] or self.rect.y < 0:
            self.rect.y -= self.dy

# 메뉴 화면
def game_menu():
    keyEnter = False
    while True:
        screen.fill((0,0,0))
        back_image = pygame.image.load('back.png')
        back_image = pygame.transform.scale(back_image, size)
        screen.blit(back_image,(0,0))
        logo_image = pygame.image.load('logo.png')
        screen.blit(logo_image,(300,50))
        
        font = pygame.font.Font(None,40)
        pressText = font.render("Press 'Enter' to start!", True, (255,255,0))
        screen.blit(pressText,(400,400))
        
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit(0)
            elif event.type==pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    keyEnter = True
            elif event.type==pygame.KEYUP:
                if event.key == pygame.K_RETURN:
                    keyEnter = False
        
        if keyEnter:
            break

# 클리어 화면
def game_clear():
    keyEnter = False
    while True:
        screen.fill((0,0,0))
        font = pygame.font.Font(None,100)
        title = font.render("Congratulations !!!", True, (255,255,255))
        title_rect = title.get_rect()
        title_rect.center = (400,80)
        screen.blit(title, title_rect.topleft)
        
        font = pygame.font.Font(None,40)
        pressText = font.render("Press 'Enter' to Quit...", True, (255,255,0))
        screen.blit(pressText,(250,380))
        
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit(0)
            elif event.type==pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    keyEnter = True
            elif event.type==pygame.KEYUP:
                if event.key == pygame.K_RETURN:
                    keyEnter = False
        
        if keyEnter:
            break


def runGame(bomb_count, coin_count, bomb_speed, coin_speed, need_score):
    # 폭탄 이미지 불러오기 and 폭탄 리스트 생성
    bomb_image = pygame.image.load('bomb.png')
    bomb_image = pygame.transform.scale(bomb_image, (50, 50))
    bombs = []
    # 코인 이미지 불러오기 and 코인 리스트 생성
    coin1_image = pygame.image.load('coin1.png')
    coin1_image = pygame.transform.scale(coin1_image, (30,30))
    coins1 = []
    coin2_image = pygame.image.load('coin2.png')
    coin2_image = pygame.transform.scale(coin2_image, (30,30))
    coins2 = []
    coin3_image = pygame.image.load('coin3.png')
    coin3_image = pygame.transform.scale(coin3_image, (30,30))
    coins3 = []
    
    # 마리오 불러오기
    mario = Mario()
    mario.load_mario()
    
    # 타이머 설정
    total_time = 60
    start_ticks=pygame.time.get_ticks()
    # 초기 라이프 설정
    total_life = 3
       
    # 스코어
    scores = 0
    
    # 현재 스코어
    string_scores = game_font.render("score  >>", True, (255,255,255))
    
    # 폭탄 개수 발생.
    for i in range(bomb_count):
        rect = pygame.Rect(bomb_image.get_rect())
        rect.left = random.randint(0, size[0]-500-29)
        rect.top = -100
        # 속도 증가
        dy = random.randint(bomb_speed//3,bomb_speed)
        bombs.append({'rect': rect, 'dy': dy})
    
    # 노란색 코인 개수 발생
    for i in range(coin_count):
        rect = pygame.Rect(coin1_image.get_rect())
        rect.left = random.randint(0, size[0]-500-69)
        rect.top = -100
        dy = random.randint(coin_speed//9,coin_speed)
        coins1.append({'rect': rect, 'dy': dy})
    
    # 빨간색 코인 개수 발생
    for i in range(coin_count//2):
        rect = pygame.Rect(coin2_image.get_rect())
        rect.left = random.randint(0, size[0]-500-69)
        rect.top = -100
        dy = random.randint(coin_speed//6,coin_speed)
        coins2.append({'rect': rect, 'dy': dy})
    
    # 파란색 코인 개수 발생
    for i in range(coin_count//4):
        rect = pygame.Rect(coin3_image.get_rect())
        rect.left = random.randint(0, size[0]-500-69)
        rect.top = -100
        dy = random.randint(coin_speed//3,coin_speed)
        coins3.append({'rect': rect, 'dy': dy})
    
    global done
    while not done:
        # 사과 이미지 띄우기
        heart_image = pygame.image.load('heart1.png')
        heart_image = pygame.transform.scale(heart_image, (30,30))
        heart = pygame.Rect(heart_image.get_rect())
        
        clock.tick(30)
        # 배경 이미지 띄우기
        bg1 = pygame.image.load("world1.png")
        bg1 = pygame.transform.scale(bg1,(600,700))
        screen.blit(bg1, (0,0))
        screen.blit(bgi, (600,0))
        
        # 경과 시간(ms)을 1000으로 나누어서 초 단위로 표시
        elapsed_time = (pygame.time.get_ticks() - start_ticks) / 1000
        # 출력할 글자, True, 글자 색상
        timer = game_font.render(str(int(total_time - elapsed_time)), True, (255,255,255))
        
        # 마리오 생성
        mario.draw_mario()
        mario.move_x()
        mario.check_screen()
        mario.update()
        
        # 타이머 문자
        string_timer = game_font.render("timer  >>", True, (255,255,255))
        
        # 타이머 띄우기
        screen.blit(timer, (1000,10))
        
        for i in range(total_life):
            screen.blit(heart_image, (700+30*i,10))
        
        # 점프 키 무한 방지
        keys = pygame.key.get_pressed()
        if(keys[pygame.K_SPACE]):
            if mario.isJump==2:
                mario.jump(1)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
                break
            # Keydown = 누르고 있을때 KeyUp 은 뗄때
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    mario.person_image = pygame.image.load('mario1.png')
                    mario.person_image = pygame.transform.scale(mario.person_image, (60, 80))
                    mario.dx = -10
                elif event.key == pygame.K_RIGHT:
                    mario.person_image = pygame.image.load('mario.png')
                    mario.person_image = pygame.transform.scale(mario.person_image, (60, 80))
                    mario.dx = 10
                elif event.key == pygame.K_SPACE:
                    if mario.isJump == 0:
                        mario.jump(1)
                    elif mario.isJump == 1:
                        mario.jump(2)
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_LEFT:
                    mario.dx = 0
                elif event.key == pygame.K_RIGHT:
                    mario.dx = 0
        
        # 땅에 닿은 폭탄 제거
        for bomb in bombs:
            bomb['rect'].top += bomb['dy']
            if bomb['rect'].top > size[1]:
                bombs.remove(bomb)
                rect = pygame.Rect(bomb_image.get_rect())
                rect.left = random.randint(0, size[0]-500-29)
                rect.top = -100
                dy = random.randint(3, 9)
                bombs.append({'rect': rect, 'dy': dy})

        # 땅에 닿은 코인 제거
        for coin1 in coins1:
            coin1['rect'].top += coin1['dy']
            if coin1['rect'].top > size[1]:
                coins1.remove(coin1)
                rect = pygame.Rect(coin1_image.get_rect())
                rect.left = random.randint(0, size[0]-500-69)
                rect.top = -100
                dy = random.randint(3,9)
                coins1.append({'rect': rect, 'dy': dy})
        
        for coin2 in coins2:
            coin2['rect'].top += coin2['dy']
            if coin2['rect'].top > size[1]:
                coins2.remove(coin2)
                rect = pygame.Rect(coin2_image.get_rect())
                rect.left = random.randint(0, size[0]-500-69)
                rect.top = -100
                dy = random.randint(3,9)
                coins2.append({'rect': rect, 'dy': dy})
        
        for coin3 in coins3:
            coin3['rect'].top += coin3['dy']
            if coin3['rect'].top > size[1]:
                coins3.remove(coin3)
                rect = pygame.Rect(coin3_image.get_rect())
                rect.left = random.randint(0, size[0]-500-69)
                rect.top = -100
                dy = random.randint(3,9)
                coins3.append({'rect': rect, 'dy': dy})
     
        # 폭탄이 닿았을때 이벤트 발생 , done = True가 되면 while not done 에서 while문 종료 즉 게임 종료
        for bomb in bombs:
            if bomb['rect'].colliderect(mario.person) and total_life > 1:
                    bombs.remove(bomb)
                    total_life -= 1
                    bomb_sound = pygame.mixer.Sound("bomb.wav")
                    bomb_sound.play()
            elif bomb['rect'].colliderect(mario.person):
                done = True
            screen.blit(bomb_image, bomb['rect'])
        # 코인 먹었을때
        for coin1 in coins1:
            if coin1['rect'].colliderect(mario.person):
                coins1.remove(coin1)
                rect = pygame.Rect(coin1_image.get_rect())
                rect.left = random.randint(0, size[0]-500-69)
                rect.top = -100
                dy = random.randint(3,9)
                coins1.append({'rect': rect, 'dy': dy})
                scores+=1
                coin_sound = pygame.mixer.Sound("coin.wav")
                coin_sound.play()
            else:
                screen.blit(coin1_image, coin1['rect'])
            
        for coin2 in coins2:
            if coin2['rect'].colliderect(mario.person):
                coins2.remove(coin2)
                rect = pygame.Rect(coin2_image.get_rect())
                rect.left = random.randint(0, size[0]-500-69)
                rect.top = -100
                dy = random.randint(3,9)
                coins2.append({'rect': rect, 'dy': dy})
                scores+=2
                coin_sound = pygame.mixer.Sound("coin.wav")
                coin_sound.play()
            else:
                screen.blit(coin2_image, coin2['rect'])
            
        for coin3 in coins3:
            if coin3['rect'].colliderect(mario.person):
                coins3.remove(coin3)
                rect = pygame.Rect(coin3_image.get_rect())
                rect.left = random.randint(0, size[0]-500-69)
                rect.top = -100
                dy = random.randint(3,9)
                coins3.append({'rect': rect, 'dy': dy})
                scores+=3
                coin_sound = pygame.mixer.Sound("coin.wav")
                coin_sound.play()
            else:
                screen.blit(coin3_image, coin3['rect'])
            
        total_scores = game_font.render(str(scores), True, (255,255,255))
        screen.blit(total_scores,(1000,50))
        screen.blit(string_scores,(850,50))
        screen.blit(string_timer,(850,10))

        # 시간 다됐을때
        if(total_time - elapsed_time<=0):
            done = True
        # 다음 스테이지
        elif(scores>=need_score):
            if(scores>=100):
                return game_clear()
            else:
                return runGame(bomb_count+2, coin_count+2, bomb_speed+5, coin_speed+5, need_score+15)
        
        pygame.display.update()

game_menu()
runGame(3,8,9,9,30)
pygame.quit()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
